In [1]:
import numpy as np # linear algebra
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import h5py

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/train_catvnoncat.hdf5', "r")
train_set_x = np.array(train_dataset["train_set_x"][:]).astype(np.float32) # your train set features
train_set_y = np.array(train_dataset["train_set_y"][:]).astype(np.float32) # your train set labels

test_dataset = h5py.File('E:/FCI/year 4/second term/NN/labs/neural-network-and-deep-learning-master/lab3/test_catvnoncat.hdf5', "r")
test_set_x= np.array(test_dataset["test_set_x"][:]).astype(np.float32) # your test set features
test_set_y = np.array(test_dataset["test_set_y"][:]).astype(np.float32) # your test set labels
classes = np.array(test_dataset["list_classes"][:]) # the list of classes
data_size = train_set_x.shape[0]


In [3]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))
print("classes = "+str(classes.shape))


train set x dim= (209, 64, 64, 3)
train set y dim= (209,)
test set x dim= (50, 64, 64, 3)
test set y dim= (50,)
classes = (2,)


In [4]:
#reshape dataset
#train data
train_set_x = train_set_x.reshape(train_set_x.shape[0],-1)
train_set_y = train_set_y.reshape(1,train_set_y.shape[0]).T
    
#test data
test_set_x = test_set_x.reshape(test_set_x.shape[0],-1)
test_set_y = test_set_y.reshape(1,test_set_y.shape[0]).T

train_set_y_n = np.where(train_set_y == 0, 1, 0)
train_set_y =  np.c_[train_set_y,train_set_y_n]

test_set_y_n = np.where(test_set_y == 0, 1, 0)
test_set_y =  np.c_[test_set_y,test_set_y_n]
print(test_set_y.shape)
print(train_set_y.shape)


(50, 2)
(209, 2)


In [5]:

#print dataset shapes(dimensions)
print("train set x dim= "+str(train_set_x.shape))
print("train set y dim= "+str(train_set_y.shape))
print("test set x dim= "+str(test_set_x.shape))
print("test set y dim= "+str(test_set_y.shape))
print("data_size = "+str(data_size))


train set x dim= (209, 12288)
train set y dim= (209, 2)
test set x dim= (50, 12288)
test set y dim= (50, 2)
data_size = 209


In [6]:
#normalize data
train_set_x=train_set_x/255
test_set_x=test_set_x/255

In [7]:

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 209
display_step = 1

In [18]:

# for visualize purpose in tensorboard we use tf.name_scope
with tf.name_scope("Declaring_placeholder"):
    # X is placeholdre for iris features. We will feed data later on
    x = tf.placeholder(tf.float32, [None, train_set_x.shape[1]])
    # y is placeholder for iris labels. We will feed data later on
    y_ = tf.placeholder(tf.float32, [None, train_set_y.shape[1]])


with tf.name_scope("Declaring_variables"):
    
    # W is our weights. This will update during training time
    W = tf.Variable(tf.random_normal([train_set_x.shape[1],train_set_y.shape[1]]))
    # b is our bias. This will also update during training time
    b = tf.Variable(tf.random_normal([train_set_y.shape[1]]))

with tf.name_scope("Declaring_functions"):
    # our prediction function
    y = tf.nn.softmax(tf.matmul(x, W) + b)


with tf.name_scope("calculating_cost"):
    # calculating cost
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y)
    cost=tf.reduce_mean(cross_entropy)



with tf.name_scope("declaring_gradient_descent"):
    # optimizer
    # we use gradient descent for our optimizer 
    train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#####
with tf.name_scope("starting_tensorflow_session"):
        # Start training
        sess = tf.InteractiveSession()
        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()
        # Run the initializer
        sess.run(init)

        # Training cycle
        for epoch in range(training_epochs):
            loss = 0.
            total_batch = int(data_size/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                pos = (i % int(data_size / batch_size)) * batch_size
                batch_xs = train_set_x[pos:pos+batch_size]
                batch_ys = train_set_y[pos:pos+batch_size]
                # Run optimization op (backprop) and cost op (to get loss value)
                _,c = sess.run([train_step,cost], feed_dict={x: batch_xs, y_: batch_ys})
                loss +=c
            print('epoch',epoch,'out of',training_epochs,'loss :',loss)
        print("Optimization Finished!")


        # Calculate accuracy
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print('train accuracy %g' % accuracy.eval(feed_dict={x: train_set_x, y_: train_set_y}))        
        print('test accuracy %g' % accuracy.eval(feed_dict={x: test_set_x, y_: test_set_y}))



epoch 0 out of 25 loss : 0.6600207686424255
epoch 1 out of 25 loss : 0.6600058674812317
epoch 2 out of 25 loss : 0.6599910259246826
epoch 3 out of 25 loss : 0.6599761843681335
epoch 4 out of 25 loss : 0.6599613428115845
epoch 5 out of 25 loss : 0.6599465012550354
epoch 6 out of 25 loss : 0.6599317193031311
epoch 7 out of 25 loss : 0.6599169969558716
epoch 8 out of 25 loss : 0.6599022746086121
epoch 9 out of 25 loss : 0.6598875522613525
epoch 10 out of 25 loss : 0.6598728895187378
epoch 11 out of 25 loss : 0.6598582863807678
epoch 12 out of 25 loss : 0.6598438024520874
epoch 13 out of 25 loss : 0.6598291993141174
epoch 14 out of 25 loss : 0.659814715385437
epoch 15 out of 25 loss : 0.6598002314567566
epoch 16 out of 25 loss : 0.6597858667373657
epoch 17 out of 25 loss : 0.6597715616226196
epoch 18 out of 25 loss : 0.6597572565078735
epoch 19 out of 25 loss : 0.6597430109977722
epoch 20 out of 25 loss : 0.6597288846969604
epoch 21 out of 25 loss : 0.6597146987915039
epoch 22 out of 25 lo